In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 32 kB/s 
     |████████████████████████████████| 198 kB 43.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=c8f310fa5142749f648b01a1bb005f34f1a2d1c54fd1327052991bf7708d17cb
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext, Row, SparkSession

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import math
import time

from itertools import combinations, product
from scipy import spatial

import heapq
from operator import itemgetter

In [ ]:
# Getting the SparkContext
conf = SparkConf().setMaster("local").setAppName("count")
sc = SparkContext.getOrCreate(conf=conf)
# Initializing the SQLContext
sqlContext = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


MLLIB

In [ ]:
# Load and parse the data rating
ratings_data = sc.textFile("drive/My Drive/Colab Notebooks/ratings.csv")
column_name = 'userId,movieId,rating,timestamp' # data.take(1)[0]
ratings = ratings_data\
    .filter(lambda l: l != column_name)\
    .map(lambda l: l.split(','))\
    .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

In [ ]:
# Load and parse the data movie
movies_data = sc.textFile("drive/My Drive/Colab Notebooks/movies.csv")
column_name = 'movieId,title,genres'
movies = movies_data\
    .filter(lambda l: l != column_name)\
    .map(lambda l: l.split(','))\
    .map(lambda l: Row(int(l[0]), l[1] ))
movies.take(1)

[<Row(1, 'Toy Story (1995)')>]

In [ ]:
# Build the recommendation model using Alternating Least Squares
regularization_parameter = 0.2
rank = 12
numIterations = 10
import math
model = ALS.train(ratings, rank, numIterations,lambda_= regularization_parameter)
# Evaluate the model on training data
testdata = ratings.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = math.sqrt(ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
print("Mean Squared Error = " + str(MSE))


Mean Squared Error = 0.7287543924410153


In [ ]:
user_id_list = [281,598]
movie_id_list = [4256,8477]
#user_product = sc.parallelize([(281,4256)])

In [ ]:
num_movie = 5
for user_id in user_id_list:
  recommendProducts_list = model.recommendProducts(user_id,num_movie)
  #print(recommendProducts_list)
  recommendProducts_list_rdd = sc.parallelize(recommendProducts_list)
  rdd1 = recommendProducts_list_rdd.map(lambda x: (x.product, x.rating))
  rdd2 = rdd1.join(movies).map(lambda x: (x[1][1],x[1][0]) )
  movie_title_list = rdd2.takeOrdered(10, key=lambda x: -x[1])
  print('*** Top', num_movie ,'movies recommended for user', user_id)
  print('\n'.join(map(str, movie_title_list)),'\n') 

*** Top 5 movies recommended for user 281
('Deathgasm (2015)', 4.748158338424845)
('On the Beach (1959)', 4.742774804526217)
('"Visitor', 4.531555542758792)
('Frozen River (2008)', 4.531555542758792)
('Saving Face (2004)', 4.517761678593577) 

*** Top 5 movies recommended for user 598
('Laggies (2014)', 4.805124957088624)
('"One I Love', 4.805124957088624)
('Delirium (2014)', 4.805124957088624)
('Deathgasm (2015)', 4.79279339341286)
('Submarine (2010)', 4.662456699686286) 



In [ ]:
num_user = 5
for movie_id in movie_id_list:
  recommendUsers_list = model.recommendUsers(movie_id,num_user)
  #print(recommendUsers_list)

  recommendUsers_rdd = sc.parallelize(recommendUsers_list)
  rdd1 = recommendUsers_rdd.map(lambda x: (x.user, x.rating))
  movie_title = movies.filter(lambda x: x[0] == movie_id).map(lambda x: (x[1],x[0])).take(1)
  user_rating_list = rdd1.takeOrdered(10, key=lambda x: -x[1])
  print('*** Top', num_user ,'users recommended for', movie_title[0])
  print('\n'.join(map(str, user_rating_list)),'\n') 

*** Top 5 users recommended for ('"Center of the World', 4256)
(53, 5.148090641540924)
(276, 4.711323432972797)
(543, 4.666315658174415)
(452, 4.659140755200704)
(243, 4.592424716358524) 

*** Top 5 users recommended for ('"Jetée', 8477)
(154, 5.891444895624094)
(53, 5.757045905836879)
(515, 5.6852908025630695)
(122, 5.635075232559433)
(171, 5.5920195148489515) 



## CF

In [ ]:
# Load and parse the data rating
test_data = sc.textFile("drive/My Drive/Colab Notebooks/cf_test.csv")
#test_data.take(1)
ratings = test_data\
    .map(lambda l: l.split(','))\
    .map(lambda l: Row(U=int(l[0]),I=int(l[1]),R=float(l[2])))

Chuẩn hóa

In [285]:
def normalized(ratings,user_set,item_set):  
  start_time = time.time()
  ## Tính mean
  # user_mean_rdd có dạng (u,mean) 
  user_mean_rdd = ratings.map(lambda x: (x[0],(x[2],1))).reduceByKey(lambda a,b: (a[0]+b[0],a[1]+b[1])).map(lambda x: (x[0],x[1][0]/x[1][1]) )


  #Lấy mỗi rating trừ đi mean
  ratings_rdd =  ratings.map(lambda x: (x[0],(x[1],x[2]))) # (u,(i,r))
  temp = ratings_rdd.join(user_mean_rdd) # join với mean
  rated_normalized_rdd = temp.mapValues(lambda x: (x[0][0],x[0][1]-x[1])).map(lambda x: ((x[0],x[1][0]),x[1][1]))

  #tìm những cặp chưa đánh giá
  a = list(product(user_set,item_set))
  rated_rdd = ratings.map(lambda x: (x[0],x[1]))
  c = set(a)-set(rated_rdd.collect())
  unrated_rdd = sc.parallelize(c)

  normalized_rdd = unrated_rdd.map(lambda x: (x,0)) + rated_normalized_rdd

  #
  x = normalized_rdd.map(lambda x: (x[0][0],x[1]))
  normalized_list = x.groupByKey().mapValues(list).map(lambda x: (x[0],set(x[1]),len(x[1]))) #(u,set,lenlist)
  print("--- %s seconds ---" % (time.time() - start_time))

  return user_mean_rdd,normalized_rdd,unrated_rdd,normalized_list

Similarity

In [286]:
def similarity(user_set,normalized_list):
  start_time = time.time()

  #hàm tính độ similarity jaccard

  def jaccard(set1,len1,set2,len2):
      intersection = len(list(set1.intersection(set2)))
      union = (len1 + len2) - intersection
      return float(intersection) / union 

  combinations_rdd = normalized_list.cartesian(normalized_list)#.filter(lambda x: x[0][0]<x[1][0])

  #tính bảng similarity
  similarity_list = combinations_rdd.map(lambda x: ((x[0][0],x[1][0]),jaccard(x[0][1],x[0][2],x[1][1],x[1][2])))#.sortBy(lambda x:x[0]).collect()

  print("--- %s seconds ---" % (time.time() - start_time))

  return similarity_list

Predict

In [291]:
def pred(uid,unrated_rdd,normalized_rdd,similarity_list,user_mean_rdd):
  start_time = time.time()
  #predict

  def predict(sim,nor):
    r =0
    s = sim
    n =nor
    if len(sim) == 2:
      r = (s[0]*n[0]+s[1]*n[1])/(abs(s[0])+abs(s[1]))
    if len(sim) == 1:
      r = (s[0]*n[0])/(abs(s[0]))
    return r  
    
  def find_sim(user_list,sim_list):
    s = list()
    for user in user_list:
      s.append(sim_list[user-1]) 
    s1= s.copy()
    s1.sort(reverse=True)
    m = s1[:2]
    q = list()
    for i in m:
      q.append(user_list[s.index(i)])
    return q,m

  def find_nor(user_list,nor_list):
    s = list()
    for user in user_list:
      s.append(nor_list[user-1]) 
    return s

  # gom các user đã đánh giá một item
  item_list_user = ratings.map(lambda x: (x[1],x[0])).groupByKey().mapValues(list)
  k =2
  #Bước 1: Tìm tất cả user đã rating item đó
  user_unrate = unrated_rdd.filter(lambda x: x[0]==uid).map(lambda x: (x[1],x[0])).join(item_list_user)\
                .map(lambda x: (x[1][0],x[0],x[1][1]))
  
  nor = normalized_rdd.map(lambda x: ((x[0][1],x[0][0]),x[1])).sortByKey().map(lambda x: (x[0][0],x[1])).groupByKey().mapValues(list)
  #chuyển sang hệ số similarity
  sim = similarity_list.map(lambda x: (x[0][0],x[1])).groupByKey().mapValues(list).filter(lambda x: x[0]==uid).values().first()

  pre = user_unrate.map(lambda x: (x[1],(x[0],find_sim(x[2],sim))) )\
        .join(nor).mapValues(lambda x: (x[0][0],x[0][1][1],find_nor(x[0][1][0],x[1])))\
        .map(lambda x: ((x[1][0],x[0]),predict(x[1][1],x[1][2])) ) 

  mean = user_mean_rdd.filter(lambda x: x[0]== uid).values().first()
  result = pre.mapValues(lambda x: x+mean)

  print("--- %s seconds ---" % (time.time() - start_time))
  return result

In [292]:
# Load and parse the data rating
ratings_data = sc.textFile("drive/My Drive/Colab Notebooks/ratings.csv")
column_name = 'userId,movieId,rating,timestamp' # data.take(1)[0]
ratings = ratings_data\
    .filter(lambda l: l != column_name)\
    .map(lambda l: l.split(','))\
    .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

#Tính tập user và tập item
user_set = set(ratings.keys().collect())
item_set = set(ratings.values().collect())

In [293]:
#def main(ratings,movies):
user_mean_rdd,normalized_rdd,unrated_rdd,normalized_list = normalized(ratings,user_set,item_set)
similarity_list = similarity(user_set,normalized_list)
uid = 3
pre = pred(uid,unrated_rdd,normalized_rdd,similarity_list,user_mean_rdd)


--- 12.072497844696045 seconds ---
--- 0.012032747268676758 seconds ---
--- 51.343868017196655 seconds ---


In [294]:
# Load and parse the data movie
movies_data = sc.textFile("drive/My Drive/Colab Notebooks/movies.csv")
column_name = 'movieId,title,genres'
movies = movies_data\
    .filter(lambda l: l != column_name)\
    .map(lambda l: l.split(','))\
    .map(lambda l: Row(int(float(l[0])), l[1] ))

num_movie = 5
#for user_id in user_id_list:
rdd1 = pre.map(lambda x: (x[0][1], x[1]))
rdd2 = rdd1.join(movies).map(lambda x: (x[1][1],x[1][0]) )
movie_title_list = rdd2.takeOrdered(num_movie, key=lambda x: -x[1])
print('*** Top', num_movie ,'movies recommended for user', uid)
print('\n'.join(map(str, movie_title_list)),'\n') 

*** Top 5 movies recommended for user 3
('"Match Factory Girl', 5.19044289044289)
('"Jetée', 5.19044289044289)
('Midnight Cowboy (1969)', 4.864468864468865)
('"Muppet Movie', 4.8285005653426705)
('"Life Less Ordinary', 4.8285005653426705) 

